In [1]:
import os
import os.path as osp
import sys
from itertools import combinations
sys.path.append(osp.abspath('..'))

import numpy as np
import scipy
from tqdm import tqdm_notebook
from scipy.stats import kurtosis, skew
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.preprocessing import scale
from sklearn.svm import LinearSVC, SVC


import config
from datasets.gtzan import GTZAN_MFCC

%load_ext autoreload
%autoreload 2

In [2]:
# Random seeds
np.random.seed(1234)
MIN_SEGMENTS = -1
TEST_SEGMENTS = -1

In [3]:
dataset = GTZAN_MFCC(phase='all')

_, C, F = dataset[0][0].shape
print('Dataset:', len(dataset))
print('Shape:', dataset[0][0].shape)

Dataset: 1000
Shape: (1, 39, 1997)


In [4]:
def concatenate(dataset):
    X = []
    Y = []
    for sample_idx, (x, y) in tqdm_notebook(enumerate(dataset), total=len(dataset)):
        x = np.hstack((
            x.mean(axis=2), 
            x.std(axis=2), 
            x.max(axis=2),
            kurtosis(x, axis=2),
            skew(x, axis=2),
        ))
        X.append(x)
        Y.append(y)
    X = np.vstack(X)
    Y = np.array(Y).flatten()
    return X, Y

X, Y = concatenate(dataset)
print(X.shape)
print(Y.shape)


(1000, 195)
(1000,)


In [5]:
scores = cross_validate(SVC(C=2), scale(X), Y, cv=10, n_jobs=-1)
print('SCORES', scores['test_score'])
print('AVG:', scores['test_score'].mean())
print('STD:', scores['test_score'].std())

SCORES [0.65 0.7  0.85 0.77 0.69 0.71 0.76 0.75 0.63 0.69]
AVG: 0.7200000000000001
STD: 0.06099180272790762


70.5%(13, Without Max)